In [1]:
# Import the necessary files and libraries that are required for operationalisation of this Project
import os
import cv2
import os
import glob
from sklearn.utils import shuffle
import numpy as np

In [2]:
# Function to change the gamma of the imagery
def adjust_gamma(image):
        gamma = 0.5
        invGamma = 1.0 / gamma
        table = np.array([((i / 255.0) ** invGamma) * 255
            for i in np.arange(0, 256)]).astype("uint8")

        return cv2.LUT(image, table)

# Function to change the brightness of the imagery
def increase_brightness(img, value):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

# Function to train the model
def load_train(train_path, image_size, classes):
    images = []
    labels = []
    img_names = []
    cls = []

    for fields in classes:   
        index = classes.index(fields)
        path = os.path.join(train_path, fields, '*g')
        files = glob.glob(path)
        for fl in files:
            image = cv2.imread(fl)

            # Get only the bottom half of the images, as this is where the road will be
            height, width = image.shape[:2]
            newHeight = int(round(height/2))
            image = image[newHeight-5:height-50, 0:width]
            brght_img = increase_brightness(image, value=150)
            shaded_img = adjust_gamma(image)
            
            # Raw image
            image = cv2.resize(image, (image_size, image_size),0,0, cv2.INTER_LINEAR)
            image = image.astype(np.float32)
            image = np.multiply(image, 1.0 / 255.0)

            # Make a version of the images that are brightened
            brght_img = cv2.resize(brght_img, (image_size, image_size),0,0, cv2.INTER_LINEAR)
            brght_img = brght_img.astype(np.float32)
            brght_img = np.multiply(brght_img, 1.0 / 255.0)

            # Make a version of the images that are shaded
            shaded_img = cv2.resize(shaded_img, (image_size, image_size),0,0, cv2.INTER_LINEAR)
            shaded_img = shaded_img.astype(np.float32)
            shaded_img = np.multiply(shaded_img, 1.0 / 255.0)
            
            # Add all three images to the dataset for training
            images.append(image)
            images.append(brght_img)
            images.append(shaded_img)

            label = np.zeros(len(classes))
            label[index] = 1.0

            for i in range(3):
                labels.append(label)    
                flbase = os.path.basename(fl)
                img_names.append(flbase)
                cls.append(fields)
           
    images = np.array(images)
    labels = np.array(labels)
    img_names = np.array(img_names)
    cls = np.array(cls)

    return images, labels, img_names, cls

# Object oriented coding to let the instance of the dataset reference itself
class DataSet(object):

  def __init__(self, images, labels, img_names, cls):
    self._num_examples = images.shape[0]

    self._images = images
    self._labels = labels
    self._img_names = img_names
    self._cls = cls
    self._epochs_done = 0
    self._index_in_epoch = 0

  @property
  def images(self):
    return self._images

  @property
  def labels(self):
    return self._labels

  @property
  def img_names(self):
    return self._img_names

  @property
  def cls(self):
    return self._cls

  @property
  def num_examples(self):
    return self._num_examples

  @property
  def epochs_done(self):
    return self._epochs_done

  def next_batch(self, batch_size):
    """Return the next `batch_size` examples from this data set."""
    start = self._index_in_epoch
    self._index_in_epoch += batch_size

    if self._index_in_epoch > self._num_examples:
      self._epochs_done += 1
      start = 0
      self._index_in_epoch = batch_size
      assert batch_size <= self._num_examples
    end = self._index_in_epoch

    return self._images[start:end], self._labels[start:end], self._img_names[start:end], self._cls[start:end]


def read_train_sets(train_path, image_size, classes, validation_size):
  class DataSets(object):
    pass
  data_sets = DataSets()

  images, labels, img_names, cls = load_train(train_path, image_size, classes)
  images, labels, img_names, cls = shuffle(images, labels, img_names, cls)  

  if isinstance(validation_size, float):
    validation_size = int(validation_size * images.shape[0])

  validation_images = images[:validation_size]
  validation_labels = labels[:validation_size]
  validation_img_names = img_names[:validation_size]
  validation_cls = cls[:validation_size]

  train_images = images[validation_size:]
  train_labels = labels[validation_size:]
  train_img_names = img_names[validation_size:]
  train_cls = cls[validation_size:]

  data_sets.train = DataSet(train_images, train_labels, train_img_names, train_cls)
  data_sets.valid = DataSet(validation_images, validation_labels, validation_img_names, validation_cls)

  return data_sets

In [3]:
# Import the necessary files and libraries that are required for operationalisation of this Project
import tensorflow as tf
import time
from datetime import timedelta
import math
import random
import numpy as np
import os

# Randomise the seed for usage latero n
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
batch_size = 32

# Prepare input data by obtaining the directory of the 8 image classes
classes = os.listdir('training_data')
num_classes = len(classes)

# 15% of the data will automatically be used for validation
validation_size = 0.15
img_size = 128
num_channels = 3
train_path='training_data'

# Load all the training and validation data into memory 
data = read_train_sets(train_path, img_size, classes, validation_size=validation_size)

# Create a tensor flow session and label in preparation for training
session = tf.Session()
x = tf.placeholder(tf.float32, shape=[None, img_size,img_size,num_channels], name='x')
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)



# Set up three convolution layers
filter_size_conv1 = 3 
num_filters_conv1 = 32

filter_size_conv2 = 3
num_filters_conv2 = 32

filter_size_conv3 = 3
num_filters_conv3 = 64

fc_layer_size = 128

def create_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

def create_biases(size):
    return tf.Variable(tf.constant(0.05, shape=[size]))

def create_convolutional_layer(input,
               num_input_channels, 
               conv_filter_size,        
               num_filters):  
    
    ## We shall define the weights that will be trained using create_weights function.
    weights = create_weights(shape=[conv_filter_size, conv_filter_size, num_input_channels, num_filters])
    ## We create biases using the create_biases function. These are also trained.
    biases = create_biases(num_filters)

    ## Creating the convolutional layer
    layer = tf.nn.conv2d(input=input, filter=weights,strides=[1, 1, 1, 1],padding='SAME')
    layer += biases
    layer = tf.nn.max_pool(value=layer,ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],padding='SAME')
    layer = tf.nn.relu(layer)

    return layer

    
# Function to flatten the shape of the data for returning an output value
def create_flatten_layer(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer = tf.reshape(layer, [-1, num_features])

    return layer

# Function to fully connect the shape of the data for returning an output value
def create_fc_layer(input,          
             num_inputs,    
             num_outputs,
             use_relu=True):
    
    # Define trainable weights and biases.
    weights = create_weights(shape=[num_inputs, num_outputs])
    biases = create_biases(num_outputs)
    layer = tf.matmul(input, weights) + biases
    if use_relu:
        layer = tf.nn.relu(layer)

    return layer

# Design the three convolution layers
layer_conv1 = create_convolutional_layer(input=x,
               num_input_channels=num_channels,
               conv_filter_size=filter_size_conv1,
               num_filters=num_filters_conv1)

layer_conv2 = create_convolutional_layer(input=layer_conv1,
               num_input_channels=num_filters_conv1,
               conv_filter_size=filter_size_conv2,
               num_filters=num_filters_conv2)

layer_conv3= create_convolutional_layer(input=layer_conv2,
               num_input_channels=num_filters_conv2,
               conv_filter_size=filter_size_conv3,
               num_filters=num_filters_conv3)
          
layer_flat = create_flatten_layer(layer_conv3)

# Design the three fully connected layers
layer_fc1 = create_fc_layer(input=layer_flat,
                     num_inputs=layer_flat.get_shape()[1:4].num_elements(),
                     num_outputs=fc_layer_size,
                     use_relu=True)

layer_fc2 = create_fc_layer(input=layer_fc1,
                     num_inputs=fc_layer_size,
                     num_outputs=num_classes,
                     use_relu=False) 

# Create a predicted dataset
y_pred = tf.nn.softmax(layer_fc2,name='y_pred')
y_pred_cls = tf.argmax(y_pred, dimension=1)
session.run(tf.global_variables_initializer())
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2, labels=y_true)
cost = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-6).minimize(cost)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


session.run(tf.global_variables_initializer()) 

# Function to show the progress of the training
def show_progress(epoch, feed_dict_train, feed_dict_validate, val_loss):
    acc = session.run(accuracy, feed_dict=feed_dict_train)
    val_acc = session.run(accuracy, feed_dict=feed_dict_validate)
    msg = "Training Epoch {0} --- Training Accuracy: {1:>6.1%}, Validation Accuracy: {2:>6.1%},  Validation Loss: {3:.3f}"
    print(msg.format(epoch + 1, acc, val_acc, val_loss))

total_iterations = 0

saver = tf.train.Saver()

# Function to support the actual training of the model for 1,000,000 iterations using a split train/test split
def train(num_iteration):
    global total_iterations
    
    for i in range(total_iterations,
                   total_iterations + num_iteration):

        x_batch, y_true_batch, _, cls_batch = data.train.next_batch(batch_size)
        x_valid_batch, y_valid_batch, _, valid_cls_batch = data.valid.next_batch(batch_size)

        
        feed_dict_tr = {x: x_batch,
                           y_true: y_true_batch}
        feed_dict_val = {x: x_valid_batch,
                              y_true: y_valid_batch}

        session.run(optimizer, feed_dict=feed_dict_tr)

        if i % int(data.train.num_examples/batch_size) == 0: 
            val_loss = session.run(cost, feed_dict=feed_dict_val)
            epoch = int(i / int(data.train.num_examples/batch_size))    
            
            show_progress(epoch, feed_dict_tr, feed_dict_val, val_loss)
            saver.save(session, './roadsurface-model') 


    total_iterations += num_iteration

train(num_iteration=1000000)    

print('\a')

Going to read training images
Now going to read 01_tarmac_good files (Index: 0)
Now going to read 02_tarmac_regular files (Index: 1)
Now going to read 03_tarmac_bad files (Index: 2)
Now going to read 04_paved_good files (Index: 3)
Now going to read 05_paved_regular files (Index: 4)
Now going to read 06_paved_bad files (Index: 5)
Now going to read 07_dirt_regular files (Index: 6)
Now going to read 08_dirt_bad files (Index: 7)
Complete reading input data. Will Now print a snippet of it
Number of files in Training-set:		21908
Number of files in Validation-set:	3865


Instructions for updating:
Use the `axis` argument instead



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Training Epoch 1 --- Training Accuracy:   3.1%, Validation Accuracy:   6.2%,  Validation Loss: 2.083
Training Epoch 2 --- Training Accuracy:  21.9%, Validation Accuracy:  25.

Training Epoch 56 --- Training Accuracy:  53.1%, Validation Accuracy:  46.9%,  Validation Loss: 1.181
Training Epoch 57 --- Training Accuracy:  53.1%, Validation Accuracy:  62.5%,  Validation Loss: 1.048
Training Epoch 58 --- Training Accuracy:  53.1%, Validation Accuracy:  68.8%,  Validation Loss: 1.267
Training Epoch 59 --- Training Accuracy:  53.1%, Validation Accuracy:  56.2%,  Validation Loss: 1.405
Training Epoch 60 --- Training Accuracy:  53.1%, Validation Accuracy:  68.8%,  Validation Loss: 0.842
Training Epoch 61 --- Training Accuracy:  53.1%, Validation Accuracy:  75.0%,  Validation Loss: 1.175
Training Epoch 62 --- Training Accuracy:  53.1%, Validation Accuracy:  84.4%,  Validation Loss: 0.852
Training Epoch 63 --- Training Accuracy:  53.1%, Validation Accuracy:  56.2%,  Validation Loss: 1.236
Training Epoch 64 --- Training Accuracy:  53.1%, Validation Accuracy:  68.8%,  Validation Loss: 0.899
Training Epoch 65 --- Training Accuracy:  53.1%, Validation Accuracy:  68.8%,  Val

Training Epoch 136 --- Training Accuracy:  75.0%, Validation Accuracy:  71.9%,  Validation Loss: 0.698
Training Epoch 137 --- Training Accuracy:  75.0%, Validation Accuracy:  81.2%,  Validation Loss: 0.617
Training Epoch 138 --- Training Accuracy:  75.0%, Validation Accuracy:  84.4%,  Validation Loss: 0.691
Training Epoch 139 --- Training Accuracy:  75.0%, Validation Accuracy:  59.4%,  Validation Loss: 0.954
Training Epoch 140 --- Training Accuracy:  75.0%, Validation Accuracy:  78.1%,  Validation Loss: 0.498
Training Epoch 141 --- Training Accuracy:  75.0%, Validation Accuracy:  78.1%,  Validation Loss: 0.724
Training Epoch 142 --- Training Accuracy:  75.0%, Validation Accuracy:  84.4%,  Validation Loss: 0.591
Training Epoch 143 --- Training Accuracy:  75.0%, Validation Accuracy:  78.1%,  Validation Loss: 0.760
Training Epoch 144 --- Training Accuracy:  75.0%, Validation Accuracy:  84.4%,  Validation Loss: 0.499
Training Epoch 145 --- Training Accuracy:  75.0%, Validation Accuracy:  8

Training Epoch 216 --- Training Accuracy:  84.4%, Validation Accuracy:  90.6%,  Validation Loss: 0.461
Training Epoch 217 --- Training Accuracy:  84.4%, Validation Accuracy:  84.4%,  Validation Loss: 0.432
Training Epoch 218 --- Training Accuracy:  84.4%, Validation Accuracy:  87.5%,  Validation Loss: 0.461
Training Epoch 219 --- Training Accuracy:  84.4%, Validation Accuracy:  75.0%,  Validation Loss: 0.608
Training Epoch 220 --- Training Accuracy:  84.4%, Validation Accuracy:  90.6%,  Validation Loss: 0.290
Training Epoch 221 --- Training Accuracy:  84.4%, Validation Accuracy:  90.6%,  Validation Loss: 0.495
Training Epoch 222 --- Training Accuracy:  84.4%, Validation Accuracy:  90.6%,  Validation Loss: 0.383
Training Epoch 223 --- Training Accuracy:  84.4%, Validation Accuracy:  87.5%,  Validation Loss: 0.505
Training Epoch 224 --- Training Accuracy:  84.4%, Validation Accuracy:  93.8%,  Validation Loss: 0.314
Training Epoch 225 --- Training Accuracy:  84.4%, Validation Accuracy:  9

Training Epoch 296 --- Training Accuracy:  87.5%, Validation Accuracy:  90.6%,  Validation Loss: 0.368
Training Epoch 297 --- Training Accuracy:  87.5%, Validation Accuracy:  87.5%,  Validation Loss: 0.305
Training Epoch 298 --- Training Accuracy:  87.5%, Validation Accuracy:  93.8%,  Validation Loss: 0.317
Training Epoch 299 --- Training Accuracy:  87.5%, Validation Accuracy:  81.2%,  Validation Loss: 0.397
Training Epoch 300 --- Training Accuracy:  87.5%, Validation Accuracy:  96.9%,  Validation Loss: 0.160
Training Epoch 301 --- Training Accuracy:  87.5%, Validation Accuracy:  96.9%,  Validation Loss: 0.379
Training Epoch 302 --- Training Accuracy:  87.5%, Validation Accuracy:  96.9%,  Validation Loss: 0.193
Training Epoch 303 --- Training Accuracy:  87.5%, Validation Accuracy:  93.8%,  Validation Loss: 0.392
Training Epoch 304 --- Training Accuracy:  87.5%, Validation Accuracy:  96.9%,  Validation Loss: 0.200
Training Epoch 305 --- Training Accuracy:  87.5%, Validation Accuracy:  9

Training Epoch 376 --- Training Accuracy:  90.6%, Validation Accuracy:  87.5%,  Validation Loss: 0.307
Training Epoch 377 --- Training Accuracy:  90.6%, Validation Accuracy:  90.6%,  Validation Loss: 0.204
Training Epoch 378 --- Training Accuracy:  90.6%, Validation Accuracy:  93.8%,  Validation Loss: 0.220
Training Epoch 379 --- Training Accuracy:  90.6%, Validation Accuracy:  87.5%,  Validation Loss: 0.286
Training Epoch 380 --- Training Accuracy:  90.6%, Validation Accuracy: 100.0%,  Validation Loss: 0.084
Training Epoch 381 --- Training Accuracy:  90.6%, Validation Accuracy:  96.9%,  Validation Loss: 0.327
Training Epoch 382 --- Training Accuracy:  90.6%, Validation Accuracy:  96.9%,  Validation Loss: 0.096
Training Epoch 383 --- Training Accuracy:  90.6%, Validation Accuracy:  96.9%,  Validation Loss: 0.338
Training Epoch 384 --- Training Accuracy:  90.6%, Validation Accuracy:  96.9%,  Validation Loss: 0.129
Training Epoch 385 --- Training Accuracy:  90.6%, Validation Accuracy:  9

KeyboardInterrupt: 

In [55]:
# Import the necessary files and libraries that are required for operationalisation of this Project
import cv2 as cv
import numpy as np
import tensorflow as tf
import argparse
import sys
import os.path
import random
import os
import glob
import operator

image_size=128
num_channels=3
images = []

# Set up details for producing the output file to be tested
outputFile = "imagery.avi" 
inputfile = r".\images\3.jpg"
# Opening frames to be created from the designated output file
cap = cv.VideoCapture(inputfile)
vid_writer = cv.VideoWriter(outputFile, cv.VideoWriter_fourcc('M','J','P','G'), 15, (round(cap.get(cv.CAP_PROP_FRAME_WIDTH)),round(cap.get(cv.CAP_PROP_FRAME_HEIGHT))))

width = int(round(cap.get(cv.CAP_PROP_FRAME_WIDTH)))
height = int(round(cap.get(cv.CAP_PROP_FRAME_HEIGHT)))
newHeight = int(round(height/2))

# Restoring the model that was trained and created as part of the previous section
sess = tf.Session()
saver = tf.train.import_meta_graph('roadsurface-model.meta')
saver.restore(sess, tf.train.latest_checkpoint('./'))

# Acessing the graph of the model created
graph = tf.get_default_graph()
y_pred = graph.get_tensor_by_name("y_pred:0")
x = graph.get_tensor_by_name("x:0")
y_true = graph.get_tensor_by_name("y_true:0")
y_test_images = np.zeros((1, len(os.listdir('training_data'))))

# Compile and classify images with the ranking of the state of the road surface quality
while cv.waitKey(1) < 0:
    hasFrame, images = cap.read()
    finalimg = images

    if not hasFrame:
        print("Classification done!")
        print("Results saved as: ", outputFile)
        cv.waitKey(3000)
        break

    images = images[newHeight-5:height-50, 0:width]
    images = cv.resize(images, (image_size, image_size), 0, 0, cv.INTER_LINEAR)
    images = np.array(images, dtype=np.uint8)
    images = images.astype('float32')
    images = np.multiply(images, 1.0/255.0)

    x_batch = images.reshape(1, image_size, image_size, num_channels)
    feed_dict_testing = {x: x_batch, y_true: y_test_images}
    result = sess.run(y_pred, feed_dict=feed_dict_testing)

    # Determine the alignment to each of the classes
    outputs = [result[0,0], result[0,1], result[0,2], result[0,3], result[0,4], result[0,5], result[0,6], result[0,7]]

    # Pick the class that has the best alignment as it'll refer to the end output and preferred classification
    value = max(outputs)
    index = np.argmax(outputs)

    # Define mapping for output/index to the output as to the road surface quality
    perfectColor = (255,0,255) #Perfect
    driveableColor = (0,255,0) #Driveable
    cautionColor = (0,255,255) #Caution
    damagedColor = (0,90,255) #Damaged
    undriveableColor = (0,0,255) #Damaged
    
    if index == 0:
        #label = 'Tarmac - Good'
        label = 'Perfect'
        prob = str("{0:.2f}".format(value))
        color = perfectColor
    elif index == 1:
        #label = 'Tarmac - Regular'
        label = 'Driveable'
        prob = str("{0:.2f}".format(value))
        color = driveableColor
    elif index == 2:
        #label = 'Tarmac - Bad'
        label = 'Caution'
        prob = str("{0:.2f}".format(value))
        color = cautionColor
    elif index == 3:
        #label = 'Paved - Good'
        label = 'Driveable'
        prob = str("{0:.2f}".format(value))
        color = driveableColor
    elif index == 4:
        #label = 'Paved - Regular'
        label = 'Driveable'
        prob = str("{0:.2f}".format(value))
        color = driveableColor
    elif index == 5:
        #label = 'Paved - Bad'
        label = 'Caution'
        prob = str("{0:.2f}".format(value))
        color = cautionColor
    elif index == 6:
        #label = 'Dirt - Regular'
        label = 'Damaged'
        prob = str("{0:.2f}".format(value))
        color = damagedColor
    elif index == 7:
        #label = 'Dirt - Bad'
        label = 'Undriveable'
        prob = str("{0:.2f}".format(value))
        color = undriveableColor 
        
    # Draw the rectangle of values highlighting whether the image dictates a good or bad road surface
    cv.rectangle(finalimg, (0, 0), (300, 40), (255, 255, 255), cv.FILLED)
    cv.putText(finalimg, 'Road Condition: ', (5,15), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)
    cv.putText(finalimg, label, (150,15), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    vid_writer.write(finalimg.astype(np.uint8))

sess.close()

INFO:tensorflow:Restoring parameters from ./roadsurface-model
[0.00086166855, 0.05278209, 0.21910238, 2.7362074e-05, 0.7125144, 0.002579258, 2.2048416e-06, 0.012130604]
Classification done!
Results saved as:  imagery.avi
